In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# import function libraries
import numpy as np
import keras
import os, sys ,math, copy
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from keras.engine import Layer, InputSpec
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from keras import initializers, regularizers, constraints
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras import backend as K
from keras.utils import np_utils

sys.setrecursionlimit(10000)

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# 파일 경로 확인 및 설정
path = r'./dataset/CNC_unid_origin_data_1209/'
path2 = r'./CNC Virtual Data set _v2'

# 샘플데이터?
train_sample = pd.read_csv(path+'train.csv', header=0, encoding='utf-8')

# 성공/실패한 아이템 갯수 확인을 위해 복사
train_sample_np = np.array(train_sample.copy())

# 총데이터 불러오기
all_files = glob.glob(path+path2+'\*.csv')
li_df = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li_df.append(df)

In [3]:
# count the number of pass/fail items

nb_pass = 0
nb_pass_half = 0
nb_defective = 0

for i in range(len(train_sample_np)):
    if train_sample_np[i,5] == 'no':
        nb_defective += 1
    if train_sample_np[i,5] == 'yes' and train_sample_np[i,6] =='yes':
        nb_pass += 1
    if train_sample_np[i,5] == 'yes' and train_sample_np[i,6] == 'no':
        nb_pass_half += 1
        
''' # pandas로 사용 시.
for i in range(len(train_sample)):
    if train_sample[train_sample.columns[5]][i] == 'no':
        nb_defective += 1
    if train_sample[train_sample.columns[5]][i] == 'yes' and train_sample[train_sample.columns[6]][i] == 'no':
        nb_pass_half += 1
    if train_sample[train_sample.columns[5]][i] == 'yes' and train_sample[train_sample.columns[6]][i] == 'yes':
        nb_pass += 1
'''

print('양품 샘플 개수 : ', nb_pass)
print('공정 마쳤으나 육안검사 미통과 샘플 개수 : ', nb_pass_half)
print('공정 중지된 샘플 개수 : ', nb_defective)
print('양품 샘플 개수 : ', nb_pass+nb_pass_half+nb_defective)

양품 샘플 개수 :  13
공정 마쳤으나 육안검사 미통과 샘플 개수 :  6
공정 중지된 샘플 개수 :  6
양품 샘플 개수 :  25


In [4]:
def tool_condition(input):
    for i in range(len(input)):
        if input[i,4] == 'unworn':
            input[i,4] = 0
        else:
            input[i,4] = 1
    return input

def item_inspection(input):
    for i in range(len(input)):
        if input[i,5] == 'no':
            input[i,6] = 2
        elif input[i,5] == 'yes' and input[i,6] == 'no':
            input[i,6] = 1
        elif input[i,5] == 'yes' and input[i,6] == 'yes':
            input[i,6] = 0
    return input


def machining_process(input):
    for i in range(len(input)):
        if input[i,47] == 'Prep':
            input[i,47] = 0
        elif input[i,47] == 'Layer 1 Up':
            input[i,47] = 1
        elif input[i,47] == 'Layer 1 Down':
            input[i,47] = 2
        elif input[i,47] == 'Layer 2 Up':
            input[i,47] = 3
        elif input[i,47] == 'Layer 2 Down':
            input[i,47] = 4
        elif input[i,47] == 'Layer 3 Up':
            input[i,47] = 5
        elif input[i,47] == 'Layer 3 Down':
            input[i,47] = 6
        elif input[i,47] == 'Repositioning':
            input[i,47] = 7
        elif input[i,47] == 'End' or input[i,47] == 'end':
            input[i,47] = 8        
        elif input[i,47] == 'Starting':
            input[i,47] = 9
    return input

In [6]:
# Modifying train.csv for training, 
# - [tool_condition]  : unworn/worn -> 0 / 1 
# - [item_inspection] : machining_finalized & passed -> yes & yes / yes & no / no : 0 / 1 / 2
# - delete 'material' column and 'No' column

train_sample_info = np.array(train_sample_np.copy())
train_sample_info = tool_condition(train_sample_info)
train_sample_info = item_inspection(train_sample_info)
print(train_sample_info)


#[feedrate  clamp_pressure  unworn/worn  label]

[[1 'aluminum' 6 4.0 0 'yes' 0]
 [2 'aluminum' 20 4.0 0 'yes' 0]
 [3 'aluminum' 6 3.0 0 'yes' 0]
 [4 'aluminum' 6 2.5 0 'no' 2]
 [5 'aluminum' 20 3.0 0 'no' 2]
 [6 'aluminum' 6 4.0 1 'yes' 1]
 [7 'aluminum' 20 4.0 1 'no' 2]
 [8 'aluminum' 20 4.0 1 'yes' 1]
 [9 'aluminum' 15 4.0 1 'yes' 1]
 [10 'aluminum' 12 4.0 1 'yes' 1]
 [11 'aluminum' 3 4.0 0 'yes' 0]
 [12 'aluminum' 3 3.0 0 'yes' 0]
 [13 'aluminum' 3 4.0 1 'yes' 0]
 [14 'aluminum' 3 3.0 1 'yes' 0]
 [15 'aluminum' 6 3.0 1 'yes' 0]
 [16 'aluminum' 20 3.0 1 'no' 2]
 [17 'aluminum' 3 2.5 0 'yes' 0]
 [18 'aluminum' 3 2.5 1 'yes' 0]
 [19 'aluminum' 15 4.0 1 'yes' 1]
 [20 'aluminum' 12 4.0 0 'no' 2]
 [21 'aluminum' 3 4.0 0 'yes' 1]
 [22 'aluminum' 20 3.0 1 'yes' 0]
 [23 'aluminum' 3 4.0 1 'no' 2]
 [24 'aluminum' 3 3.0 0 'yes' 0]
 [25 'aluminum' 6 2.5 1 'yes' 0]]


In [7]:
train_sample_info = np.delete(train_sample_info,5,1)
train_sample_info = np.delete(train_sample_info,0,1)
train_sample_info = np.delete(train_sample_info,0,1)
print(train_sample_info)

[[6 4.0 0 0]
 [20 4.0 0 0]
 [6 3.0 0 0]
 [6 2.5 0 2]
 [20 3.0 0 2]
 [6 4.0 1 1]
 [20 4.0 1 2]
 [20 4.0 1 1]
 [15 4.0 1 1]
 [12 4.0 1 1]
 [3 4.0 0 0]
 [3 3.0 0 0]
 [3 4.0 1 0]
 [3 3.0 1 0]
 [6 3.0 1 0]
 [20 3.0 1 2]
 [3 2.5 0 0]
 [3 2.5 1 0]
 [15 4.0 1 1]
 [12 4.0 0 2]
 [3 4.0 0 1]
 [20 3.0 1 0]
 [3 4.0 1 2]
 [3 3.0 0 0]
 [6 2.5 1 0]]


In [12]:
k  = 0
li_pass = []
li_pass_half = []
li_fail = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)      
    
    if train_sample_info[k,3] == 0:
        li_pass.append(df)        
    elif train_sample_info[k,3] == 1:
        li_pass_half.append(df)        
    else :
        li_fail.append(df)
        
    k += 1
    
frame01 = pd.concat(li_pass, axis=0, ignore_index=True)
frame02 = pd.concat(li_pass_half, axis=0, ignore_index=True)
frame03 = pd.concat(li_fail, axis=0, ignore_index=True)

data_pass = np.array(frame01.copy())
data_pass_half = np.array(frame02.copy())
data_fail = np.array(frame03.copy())


print('공정완료 및 육안검사 합격한 전체 데이터 수 : ',len(data_pass))
print('공정완료 및 육안검사 불합격한 전체 데이터 수 : ',len(data_pass_half))
print('공정 미완료한 전체 데이터 수 : ',len(data_fail))

공정완료 및 육안검사 합격한 전체 데이터 수 :  22645
공정완료 및 육안검사 불합격한 전체 데이터 수 :  6175
공정 미완료한 전체 데이터 수 :  3228


In [13]:
# Modifying experiment data 
#  - machining_process : From "Prep" to "End" -> 0~9
data_pass = machining_process(data_pass)
data_pass_half = machining_process(data_pass_half)
data_fail = machining_process(data_fail)

In [10]:
data_pass = data_pass.astype('float32')
data_pass_half = data_pass_half.astype('float32')
data_fail = data_fail.astype('float32')

data_pass

array([[ 202.    ,    4.    ,    4.    , ...,    0.    ,   50.    ,
           9.    ],
       [ 202.    ,   -6.8   , -346.    , ...,    4.    ,   50.    ,
           0.    ],
       [ 200.    ,  -13.8   ,   -2.25  , ...,    7.    ,   50.    ,
           0.    ],
       ...,
       [ 178.5   ,    3.5475,   17.5   , ...,    0.    ,   20.    ,
           8.    ],
       [ 178.5   ,    3.4475,   11.25  , ...,    0.    ,   20.    ,
           8.    ],
       [ 178.    ,    3.5625,    1.85  , ...,    0.    ,   20.    ,
           8.    ]], dtype=float32)

In [14]:
data_pass_df = pd.DataFrame(data_pass, columns=frame01.columns)
data_pass_half_df = pd.DataFrame(data_pass_half, columns=frame01.columns)
data_fail_df = pd.DataFrame(data_fail, columns=frame01.columns)

data_pass_df

,X_ActualPosition,X_ActualVelocity,X_ActualAcceleration,X_SetPosition,X_SetVelocity,X_SetAcceleration,X_CurrentFeedback,X_DCBusVoltage,X_OutputCurrent,X_OutputVoltage,...,S_CurrentFeedback,S_DCBusVoltage,S_OutputCurrent,S_OutputVoltage,S_OutputPower,S_SystemInertia,M_CURRENT_PROGRAM_NUMBER,M_sequence_number,M_CURRENT_FEEDRATE,Machining_Process
0,202,4,4,202,4,4,0.18,0.0207,329,2.77,...,0.524,2.74e-19,329,0,6.96e-07,16,1,0,50,9
1,202,-6.8,-346,202,-9.6,-354,-10.9,0.186,328,23.3,...,-0.288,2.74e-19,328,0,-5.27e-07,16,1,4,50,0
2,200,-13.8,-2.25,200,-13.9,3.9999,-8.59,0.14,328,30.6,...,0.524,2.74e-19,328,0,9.1e-07,16,1,7,50,0
3,198,-14,4,198,-13.9,3.9999,-6.11,0.13,327,30.3,...,-0.18,2.74e-19,328,0,1.07e-06,16,1,7,50,0
4,197,-13.9,-14.8,196,-13.9,4.0001,-5.7,0.114,328,30.5,...,0.226,2.74e-19,328,0,1.96e-06,16,1,7,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22640,179,3.5625,8.15,179,3.5,5,-0.941,0.0215,328,1.79,...,0.244,2.77e-19,328,0,-3.02e-06,17,1,0,20,8
22641,178.5,3.535,30,178.5,3.5,5,-0.78,0.0202,328,1.07,...,0.244,2.77e-19,328,0,0,17,1,0,20,8
22642,178.5,3.5475,17.5,178.5,3.5,5,0.501,0.019,328,0.99,...,0.128,2.77e-19,328,0,0,17,1,0,20,8
22643,178.5,3.4475,11.25,178.5,3.5,5,-0.298,0.0193,328,1.37,...,0.657,2.77e-19,328,0,-7.98e-06,17,1,0,20,8


In [15]:
data_pass_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22645 entries, 0 to 22644
Data columns (total 48 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   X_ActualPosition          22645 non-null  object
 1   X_ActualVelocity          22645 non-null  object
 2   X_ActualAcceleration      22645 non-null  object
 3   X_SetPosition             22645 non-null  object
 4   X_SetVelocity             22645 non-null  object
 5   X_SetAcceleration         22645 non-null  object
 6   X_CurrentFeedback         22645 non-null  object
 7   X_DCBusVoltage            22645 non-null  object
 8   X_OutputCurrent           22645 non-null  object
 9   X_OutputVoltage           22645 non-null  object
 10  X_OutputPower             22645 non-null  object
 11  Y_ActualPosition          22645 non-null  object
 12  Y_ActualVelocity          22645 non-null  object
 13  Y_ActualAcceleration      22645 non-null  object
 14  Y_SetPosition         

In [16]:
data_pass_df.columns

Index(['X_ActualPosition', 'X_ActualVelocity', 'X_ActualAcceleration',
       'X_SetPosition', 'X_SetVelocity', 'X_SetAcceleration',
       'X_CurrentFeedback', 'X_DCBusVoltage', 'X_OutputCurrent',
       'X_OutputVoltage', 'X_OutputPower', 'Y_ActualPosition',
       'Y_ActualVelocity', 'Y_ActualAcceleration', 'Y_SetPosition',
       'Y_SetVelocity', 'Y_SetAcceleration', 'Y_CurrentFeedback',
       'Y_DCBusVoltage', 'Y_OutputCurrent', 'Y_OutputVoltage', 'Y_OutputPower',
       'Z_ActualPosition', 'Z_ActualVelocity', 'Z_ActualAcceleration',
       'Z_SetPosition', 'Z_SetVelocity', 'Z_SetAcceleration',
       'Z_CurrentFeedback', 'Z_DCBusVoltage', 'Z_OutputCurrent',
       'Z_OutputVoltage', 'S_ActualPosition', 'S_ActualVelocity',
       'S_ActualAcceleration', 'S_SetPosition', 'S_SetVelocity',
       'S_SetAcceleration', 'S_CurrentFeedback', 'S_DCBusVoltage',
       'S_OutputCurrent', 'S_OutputVoltage', 'S_OutputPower',
       'S_SystemInertia', 'M_CURRENT_PROGRAM_NUMBER', 'M_sequence_n

In [21]:
for i in range(len(data_pass_df.columns)):
    print(len(data_pass_df[data_pass_df.columns[i]].unique()))

97
1300
567
97
2178
2562
2150
1005
10
1547
4066
655
1297
587
654
2179
2519
2722
1048
13
1820
4931
493
378
287
508
85
50
1
1
1
1
3619
419
2832
3626
61
17
944
443
34
88
1580
3
3
133
4
10


# 마지막의 컬럼 중 -1, -2, -4 인
# 'M_CURRENT_PROGRAM_NUMBER', 'M_CURRENT_FEEDRATE', 'Machining_Process'
# 는 범주형변수로 취급하고, 레이블인코딩이나 원핫인코딩을 해줘야할 것 같다.
# https://ysyblog.tistory.com/71

In [28]:
data_pass_df2 = pd.get_dummies(data_pass_df, columns=['M_CURRENT_PROGRAM_NUMBER','M_CURRENT_FEEDRATE','Machining_Process'])
data_pass_half_df2 = pd.get_dummies(data_pass_half_df, columns=['M_CURRENT_PROGRAM_NUMBER','M_CURRENT_FEEDRATE','Machining_Process'])
data_fail_df2 = pd.get_dummies(data_fail_df, columns=['M_CURRENT_PROGRAM_NUMBER','M_CURRENT_FEEDRATE','Machining_Process'])

In [29]:
data_pass_df2

,X_ActualPosition,X_ActualVelocity,X_ActualAcceleration,X_SetPosition,X_SetVelocity,X_SetAcceleration,X_CurrentFeedback,X_DCBusVoltage,X_OutputCurrent,X_OutputVoltage,...,Machining_Process_0,Machining_Process_1,Machining_Process_2,Machining_Process_3,Machining_Process_4,Machining_Process_5,Machining_Process_6,Machining_Process_7,Machining_Process_8,Machining_Process_9
0,202,4,4,202,4,4,0.18,0.0207,329,2.77,...,0,0,0,0,0,0,0,0,0,1
1,202,-6.8,-346,202,-9.6,-354,-10.9,0.186,328,23.3,...,1,0,0,0,0,0,0,0,0,0
2,200,-13.8,-2.25,200,-13.9,3.9999,-8.59,0.14,328,30.6,...,1,0,0,0,0,0,0,0,0,0
3,198,-14,4,198,-13.9,3.9999,-6.11,0.13,327,30.3,...,1,0,0,0,0,0,0,0,0,0
4,197,-13.9,-14.8,196,-13.9,4.0001,-5.7,0.114,328,30.5,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22640,179,3.5625,8.15,179,3.5,5,-0.941,0.0215,328,1.79,...,0,0,0,0,0,0,0,0,1,0
22641,178.5,3.535,30,178.5,3.5,5,-0.78,0.0202,328,1.07,...,0,0,0,0,0,0,0,0,1,0
22642,178.5,3.5475,17.5,178.5,3.5,5,0.501,0.019,328,0.99,...,0,0,0,0,0,0,0,0,1,0
22643,178.5,3.4475,11.25,178.5,3.5,5,-0.298,0.0193,328,1.37,...,0,0,0,0,0,0,0,0,1,0


In [50]:
data_fail_df2

,X_ActualPosition,X_ActualVelocity,X_ActualAcceleration,X_SetPosition,X_SetVelocity,X_SetAcceleration,X_CurrentFeedback,X_DCBusVoltage,X_OutputCurrent,X_OutputVoltage,...,M_CURRENT_FEEDRATE_50,Machining_Process_0,Machining_Process_1,Machining_Process_2,Machining_Process_3,Machining_Process_4,Machining_Process_5,Machining_Process_6,Machining_Process_7,Machining_Process_8
0,202,4.025,10.25,202,4,4,-1.42,0.0219,328,0.557,...,1,1,0,0,0,0,0,0,0,0
1,202,4,4,202,4,4,-1.9,0.0206,329,1.92,...,1,1,0,0,0,0,0,0,0,0
2,202,4,4,202,4,4,-2.06,0.0237,329,1.81,...,1,1,0,0,0,0,0,0,0,0
3,202,4.025,10.25,202,4,4,-2.22,0.0225,329,1.87,...,1,1,0,0,0,0,0,0,0,0
4,202,3.975,-2.25,202,4,4,0.499,0.026,329,1.79,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,154,9.9,19.65,154,9.95,6.92,0.555,0.0226,327,2.22,...,1,0,0,0,0,0,0,0,0,1
3224,154.5,9.65,22.75,154.5,10,4,1.67,0.0268,327,3,...,1,0,0,0,0,0,0,0,0,1
3225,155,9.875,-64.6,155,10,4,0.544,0.0274,327,1.51,...,1,0,0,0,0,0,0,0,0,1
3226,155.5,9.95,-52.125,155.5,10,4,1.1,0.0275,327,1.9,...,1,0,0,0,0,0,0,0,0,1


In [31]:
data_pass2 = np.array(data_pass_df2)
data_pass_half2 = np.array(data_pass_half_df2)
data_fail2 =np.array(data_fail_df2)

In [38]:
a = 6175*6/10
b = 3228*6/10
c = 6175*4/10
d = 3228*4/10
print('훈련셋 수')
print(a,'+',b,'=',a+b)
print(a)
print(b)
print()
print('테스트셋 수')
print(c,'+',d,'=',c+d)
print(c)
print(d)

훈련셋 수
3705.0 + 1936.8 = 5641.8
3705.0
1936.8

테스트셋 수
2470.0 + 1291.2 = 3761.2
2470.0
1291.2


In [43]:
# label 0/1 --> data01 / data02+data03

data01 = data_pass2[0:(1937+3705)*2,:]
data02 = data_pass_half2[0:3705,:]
data03 = data_fail2[0:1937,:]

train_data = np.concatenate((data01,data02),axis=0)
train_data = np.concatenate((train_data,data03),axis=0)

data04 = data_pass2[(1937+3705)*2:(1937+1291+3705+2470)*2,:]
data05 = data_pass_half2[3705:3705+2470,:]
data06 = data_fail2[1937:1937+1291,:]

test_data = np.concatenate((data04,data05),axis=0)
test_data = np.concatenate((test_data,data06),axis=0)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 62 and the array at index 1 has size 58

In [44]:
data01 = data_pass2[0:(1937+3705)*2,:]
data02 = data_pass_half2[0:3705,:]
data03 = data_fail2[0:1937,:]

train_data = np.concatenate((data01,data02),axis=0)

In [46]:
train_data.shape

(14989, 62)

In [47]:
data03.shape

(1937, 58)

In [48]:
data02.shape

(3705, 62)

In [49]:
data01.shape

(11284, 62)